**Feature Engineering**

**простая агрегированная статистика qty не может быть полноценно использована в качестве признака или таргета для модели.** Если у нас есть только информация об активности товара за сегодняшний день, это мало что говорит о том, что будет с этим товаром завтра или через неделю.

Нам необходимо более эффективно использовать имеющуюся историческую информацию.

Как посчитать нужные нам признаки и целевые переменные?
Оконная функция .rolling() в pandas создает "скользящее окно" строк в датафрейме, к которому затем можно применять другие методы. Все последующие методы, такие как сумма или среднее, работают уже с этим окном, выполняя вычисления для каждого подмножества данных. **Цепочки методов в pandas** – это **последовательный вызов различных методов над датафреймом**, где результат одного метода передается следующему методу в цепочке.

Для каждой строки датасета (строка — это товар-день) в качестве признаков мы рассчитаем:

1. среднее количество продаж за последние N дней,
2. Квантиль X продаж за последние N дней.

Для каждой строки датасета (строка — это товар-день) в качестве признаков мы рассчитаем:

1. среднее количество продаж за последние N дней,
2. Квантиль X продаж за последние N дней.
В качестве таргетов мы рассчитаем:

1. суммарное количество продаж за следующие N дней.

**add_features** – добавляет в DataFrame новые признаки, вычисленные по методу скользящего окна для каждого sku_id за последние N дней.

**add_targets** – добавляет в DataFrame новые целевые переменные, рассчитанные как сумма следующих N дней для каждого sku_id, исключая текущий день.


1. **qty_7d_avg, qty_14d_avg, qty_21d_avg**– среднее количество продаж за предыдущие 1, 2 и 3 недели, включая сегодняшний день.
2. **qty_7d_q10, qty_7d_q50, qty_7d_q90** (аналогично 14d, 21d) – **квантили 0.1, 0.5, 0.9 продаж за предыдущие 1, 2, 3 недели.**
3. **next_7d, next_14d, next_21d** – суммарное количество продаж за следующие 1, 2, 3 недели, не включая сегодняшний день (таргеты).

In [ ]:
FEATURES = {
    "qty_7d_avg": ("qty", 7, "avg", None),
    "qty_7d_q10": ("qty", 7, "quantile", 10),
    "qty_7d_q50": ("qty", 7, "quantile", 50),
    "qty_7d_q90": ("qty", 7, "quantile", 90),
    "qty_14d_avg": ("qty", 14, "avg", None),
    "qty_14d_q10": ("qty", 14, "quantile", 10),
    "qty_14d_q50": ("qty", 14, "quantile", 50),
    "qty_14d_q90": ("qty", 14, "quantile", 90),
    "qty_21d_avg": ("qty", 21, "avg", None),
    "qty_21d_q10": ("qty", 21, "quantile", 10),
    "qty_21d_q50": ("qty", 21, "quantile", 50),
    "qty_21d_q90": ("qty", 21, "quantile", 90),
}

TARGETS = {
    #"next_2d": ("qty", 2),
    "next_7d": ("qty", 7),
    "next_14d": ("qty", 14),
    "next_21d": ("qty", 21),
}

In [ ]:
import pandas as pd
from typing import Dict, Tuple, Optional


def add_features(
    df: pd.DataFrame,
    features: Dict[str, Tuple[str, int, str, Optional[int]]],
) -> None:
    """
    Add rolling features to the DataFrame based on the specified aggregations.
    For each sku_id, the features are computed as the aggregations of the last N-days.
    Current date is always included into rolling window.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame to add the feature to. Changes are applied inplace.
    features : Dict[str, Tuple[str, int, str, Optional[int]]]
        Dictionary with the following structure:
        {
            "feature_name": ("agg_col", "days", "aggregation_function", "quantile"),
            ...
        }
        where:
            - feature_name: name of the feature to add
            - agg_col: name of the column to aggregate
            - int: number of days to include into rolling window
            - aggregation_function: one of the following: "quantile", "avg"
            - int: quantile to compute (only for "quantile" aggregation_function)

    Raises
    ------
    ValueError
        If aggregation_function is not one of the following: "quantile", "avg"
    """
    for feature_name, (agg_col, days, agg_func, quantile) in features.items():
        if agg_func == "quantile":
            df[feature_name] = (
                df.groupby("sku_id")[agg_col]
                .rolling(window=days)
                .quantile(quantile / 100)
                .reset_index(level=0, drop=True)
            )
        elif agg_func == "avg":
            df[feature_name] = (
                df.groupby("sku_id")[agg_col]
                .rolling(window=days)
                .mean()
                .reset_index(level=0, drop=True)
            )
        else:
            raise ValueError(f"Unknown aggregation function: {agg_func}")

def add_targets(df: pd.DataFrame, targets: Dict[str, Tuple[str, int]]) -> None:
    df.sort_values(['sku_id', 'day'], inplace=True)
    for target_name, (agg_col, days) in targets.items():
        res = (
            df.groupby("sku_id")[agg_col]
              .shift(-1)
              .rolling(window=days, min_periods=days)
              .sum()
              .shift(-(days - 1))  # <-- тут магия!
              .reset_index(level=0, drop=True)
        )
        df[target_name] = res

In [ ]:
def add_targets(df: pd.DataFrame, targets: Dict[str, Tuple[str, int]]) -> None:
    df.sort_values(['sku_id', 'day'], inplace=True)
    for target_name, (agg_col, days) in targets.items():
        res = (
            df.groupby("sku_id")[agg_col]
              .shift(-1)
              .rolling(window=days, min_periods=days)
              .sum()
              .shift(-(days - 1))  # <-- тут магия!
              .reset_index(level=0, drop=True)
        )
        df[target_name] = res

In [ ]:
df = pd.read_csv('/content/query.csv')
df

,day,sku_id,sku,price,qty
0,2018-07-01,0,Router 64,1460,7
1,2018-07-02,0,Router 64,1460,4
2,2018-07-03,0,Router 64,1460,0
3,2018-07-04,0,Router 64,1460,5
4,2018-07-05,0,Router 64,1460,6
...,...,...,...,...,...
112492,2022-06-26,78,Microphone 391,1650,11
112493,2022-06-27,78,Microphone 391,1650,17
112494,2022-06-28,78,Microphone 391,1650,12
112495,2022-06-29,78,Microphone 391,1650,15


In [ ]:
import time
start = time.time()
add_targets(df,TARGETS)
add_features(df,FEATURES)
end = time.time()
print(f"Время выполнения: {end} сек")

Время выполнения: 1749409989.7007573 сек


In [ ]:
df.to_csv('features.csv', index=False)

Задача регрессии, когда необходимо предсказывать не средние значения таргета, а квантили, называется квантильной регрессией.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/features.csv')
df

,day,sku_id,sku,price,qty,next_7d,next_14d,next_21d,qty_7d_avg,qty_7d_q10,qty_7d_q50,qty_7d_q90,qty_14d_avg,qty_14d_q10,qty_14d_q50,qty_14d_q90,qty_21d_avg,qty_21d_q10,qty_21d_q50,qty_21d_q90
0,2018-07-01,0,Router 64,1460,7,60.0,69.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-02,0,Router 64,1460,4,60.0,65.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-07-03,0,Router 64,1460,0,65.0,68.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-04,0,Router 64,1460,5,60.0,66.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-05,0,Router 64,1460,6,54.0,62.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112492,2022-06-26,78,Microphone 391,1650,11,NaN,NaN,NaN,11.428571,4.2,11.0,19.6,10.071429,0.0,10.0,18.2,7.571429,0.0,7.0,14.0
112493,2022-06-27,78,Microphone 391,1650,17,NaN,NaN,NaN,12.857143,4.2,13.0,21.4,11.285714,2.1,11.0,19.1,8.380952,0.0,7.0,17.0
112494,2022-06-28,78,Microphone 391,1650,12,NaN,NaN,NaN,12.571429,4.2,12.0,21.4,11.571429,2.1,11.5,19.1,8.952381,0.0,8.0,17.0
112495,2022-06-29,78,Microphone 391,1650,15,NaN,NaN,NaN,14.714286,9.4,13.0,21.4,12.642857,7.0,12.5,19.1,9.666667,0.0,9.0,17.0


In [ ]:
import pandas as pd
from typing import List, Tuple, Dict
import numpy as np
import pandas as pd
from sklearn.linear_model import QuantileRegressor
from tqdm import tqdm

def split_train_test(
    df: pd.DataFrame,
    test_days: int = 30,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df = df.copy()
    df = df.dropna()
    max_day = df["day"].max()
    split_day = max_day - pd.Timedelta(days=(test_days+1))
    df_test = df[df["day"] > split_day]
    df_train = df[df["day"] <= split_day]
    return df_train, df_test

class MultiTargetModel:
    def __init__(
        self,
        features: List[str],
        horizons: List[int] = [7, 14, 21],
        quantiles: List[float] = [0.1, 0.5, 0.9],
    ) -> None:
        self.quantiles = quantiles
        self.horizons = horizons
        self.sku_col = "sku_id"
        self.date_col = "day"
        self.features = features
        self.targets = [f"next_{horizon}d" for horizon in self.horizons]
        self.fitted_models_ = {}

    def fit(self, data: pd.DataFrame, verbose: bool = False) -> None:
        """Fit model on data."""
        sku_ids = data[self.sku_col].unique()

        iterable = tqdm(sku_ids) if verbose else sku_ids

        for sku_id in iterable:
            sku_data = data[data[self.sku_col] == sku_id]
            if len(sku_data) == 0:
                continue

            self.fitted_models_[sku_id] = {}

            X = sku_data[self.features].values
            for horizon in self.horizons:
                y = sku_data[f"next_{horizon}d"].values

                for quantile in self.quantiles:
                    model = QuantileRegressor(
                        quantile=quantile,
                        alpha=1.0,
                        solver='highs'
                    )
                    model.fit(X, y)
                    self.fitted_models_[sku_id][(quantile, horizon)] = model

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        """Predict on data."""
        predictions = data[[self.sku_col, self.date_col]].copy()

        # Initialize all prediction columns with 0
        for horizon in self.horizons:
            for quantile in self.quantiles:
                predictions[f"pred_{horizon}d_q{int(quantile*100)}"] = 0.0

        # Make predictions for known SKUs
        for sku_id in data[self.sku_col].unique():
            if sku_id in self.fitted_models_:
                sku_data = data[data[self.sku_col] == sku_id]
                X = sku_data[self.features].values

                for (quantile, horizon), model in self.fitted_models_[sku_id].items():
                    pred_col = f"pred_{horizon}d_q{int(quantile*100)}"
                    predictions.loc[predictions[self.sku_col] == sku_id, pred_col] = model.predict(X)

        return predictions

def quantile_loss(y_true: np.ndarray, y_pred: np.ndarray, quantile: float) -> float:
    """
    Calculate the quantile loss between the true and predicted values.
    """
    errors = y_true - y_pred
    loss = np.mean(np.maximum(quantile * errors, (quantile - 1) * errors))
    return loss

Разделить датасет на train и test (30 последних дней)
(простое разделение без кросс-валидации)

Для каждого sku:
    Для каждого горизонта планирования (1, 2, 3 недели):
        Для каждого квантиля (0.1, 0.5, 0.9):
            Обучить модель на train
            Сохранить модель в словарь

Для каждого sku:
    Для каждого горизонта планирования (1, 2, 3 недели):
        Для каждого квантиля (0.1, 0.5, 0.9):
            Сделать прогноз для test и сохранить результаты в датафрейм

Оценить качество модели с помощью метрики

In [ ]:
df['day'] = pd.to_datetime(df['day'])

In [ ]:
df_train, df_test = split_train_test(df, test_days=30)

In [ ]:
model = MultiTargetModel(
    features=[
        "price",
        "qty",
        "qty_7d_avg",
        "qty_7d_q10",
        "qty_7d_q50",
        "qty_7d_q90",
        "qty_14d_avg",
        "qty_14d_q10",
        "qty_14d_q50",
        "qty_14d_q90",
        "qty_21d_avg",
        "qty_21d_q10",
        "qty_21d_q50",
        "qty_21d_q90",
    ],
    horizons=[7, 14, 21],
    quantiles=[0.1, 0.5, 0.9],
)
model.fit(df_train, verbose=True)

predictions = model.predict(df_test)

100%|██████████| 77/77 [01:08<00:00,  1.12it/s]


In [ ]:
predictions.columns

Index(['sku_id', 'day', 'pred_7d_q10', 'pred_7d_q50', 'pred_7d_q90',
       'pred_14d_q10', 'pred_14d_q50', 'pred_14d_q90', 'pred_21d_q10',
       'pred_21d_q50', 'pred_21d_q90'],
      dtype='object')

In [ ]:
df.columns

Index(['day', 'sku_id', 'sku', 'price', 'qty', 'next_7d', 'next_14d',
       'next_21d', 'qty_7d_avg', 'qty_7d_q10', 'qty_7d_q50', 'qty_7d_q90',
       'qty_14d_avg', 'qty_14d_q10', 'qty_14d_q50', 'qty_14d_q90',
       'qty_21d_avg', 'qty_21d_q10', 'qty_21d_q50', 'qty_21d_q90'],
      dtype='object')

In [ ]:
import pandas as pd


def evaluate_model(
    df_true: pd.DataFrame,
    df_pred: pd.DataFrame,
    quantiles: List[float] = [0.1, 0.5, 0.9],
    horizons: List[int] = [7, 14, 21],
) -> pd.DataFrame:
    """Evaluate model on data.

    Parameters
    ----------
    df_true : pd.DataFrame
        True values.
    df_pred : pd.DataFrame
        Predicted values.
    quantiles : List[float], optional
        Quantiles to evaluate on, by default [0.1, 0.5, 0.9].
    horizons : List[int], optional
        Horizons to evaluate on, by default [7, 14, 21].

    Returns
    -------
    pd.DataFrame
        Evaluation results.
    """
    losses = {}

    for quantile in quantiles:
        for horizon in horizons:
            true = df_true[f"next_{horizon}d"].values
            pred = df_pred[f"pred_{horizon}d_q{int(quantile*100)}"].values
            loss = quantile_loss(true, pred, quantile)

            losses[(quantile, horizon)] = loss

    losses = pd.DataFrame(losses, index=["loss"]).T.reset_index()
    losses.columns = ["quantile", "horizon", "avg_quantile_loss"]  # type: ignore

    return losses

In [ ]:
# df_test — это твои настоящие значения на тесте
# df_pred — это предсказания модели на этих объектах
result = evaluate_model(df_test, predictions)
result

,quantile,horizon,avg_quantile_loss
0,0.1,7,1.734478
1,0.1,14,3.338710
2,0.1,21,5.058609
3,0.5,7,5.174562
4,0.5,14,9.826750
5,0.5,21,14.799232
6,0.9,7,5.526991
7,0.9,14,10.007423
8,0.9,21,14.038607


**Бутстрэп (Bootstrap)** — это статистический метод, позволяющий оценить величину ошибки и уверенность в ней при использовании оценочных значений. С помощью бутсрэпа формируют доверительный интервал, который и является оценкой величины ошибки.

Наша задача заключается в том, чтобы 80% значений попали в доверительный интервал.

In [ ]:
df

,day,sku_id,sku,price,qty,next_7d,next_14d,next_21d,qty_7d_avg,qty_7d_q10,qty_7d_q50,qty_7d_q90,qty_14d_avg,qty_14d_q10,qty_14d_q50,qty_14d_q90,qty_21d_avg,qty_21d_q10,qty_21d_q50,qty_21d_q90
0,2018-07-01,0,Router 64,1460,7,60.0,69.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-02,0,Router 64,1460,4,60.0,65.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-07-03,0,Router 64,1460,0,65.0,68.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-04,0,Router 64,1460,5,60.0,66.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-05,0,Router 64,1460,6,54.0,62.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112492,2022-06-26,78,Microphone 391,1650,11,NaN,NaN,NaN,11.428571,4.2,11.0,19.6,10.071429,0.0,10.0,18.2,7.571429,0.0,7.0,14.0
112493,2022-06-27,78,Microphone 391,1650,17,NaN,NaN,NaN,12.857143,4.2,13.0,21.4,11.285714,2.1,11.0,19.1,8.380952,0.0,7.0,17.0
112494,2022-06-28,78,Microphone 391,1650,12,NaN,NaN,NaN,12.571429,4.2,12.0,21.4,11.571429,2.1,11.5,19.1,8.952381,0.0,8.0,17.0
112495,2022-06-29,78,Microphone 391,1650,15,NaN,NaN,NaN,14.714286,9.4,13.0,21.4,12.642857,7.0,12.5,19.1,9.666667,0.0,9.0,17.0


In [ ]:
import pandas as pd

def week_missed_profits(
    df: pd.DataFrame,
    sales_col: str,
    forecast_col: str,
    date_col: str = "day",
    price_col: str = "price",
) -> pd.DataFrame:
    """
    Calculates the missed profits every week for the given DataFrame.
    """
    # Для удобства преобразуем дату к неделе
    df = df.copy()
    df["week"] = pd.to_datetime(df[date_col]).dt.to_period("W-SUN").dt.end_time
    df["week"] = df["week"].dt.normalize()
    # Посчитаем missed_sold per row
    df["missed_sold"] = (df[forecast_col] - df[sales_col]).clip(lower=0)
    df["missed_profits"] = df["missed_sold"] * df[price_col]

    # revenue - реальная выручка
    df["revenue"] = df[sales_col] * df[price_col]

    # Группируем по неделям
    res = df.groupby("week").agg(
        revenue=("revenue", "sum"),
        missed_profits=("missed_profits", "sum")
    ).reset_index().rename(columns={"week": "day"})
    return res

In [ ]:
df = df.dropna()

In [ ]:
# Для 21 дня и квантиль 90
df_merged = pd.merge(
    df,
    predictions,
    on=['sku_id', 'day'],
    how='inner'
)

In [ ]:
df_merged

,day,sku_id,sku,price,qty,next_7d,next_14d,next_21d,qty_7d_avg,qty_7d_q10,...,qty_21d_q90,pred_7d_q10,pred_7d_q50,pred_7d_q90,pred_14d_q10,pred_14d_q50,pred_14d_q90,pred_21d_q10,pred_21d_q50,pred_21d_q90
0,2022-05-10,0,Router 64,1460,0,33.0,76.0,93.0,2.142857,0.0,...,7.0,10.0,30.000000,59.0,32.0,65.0,102.0,61.0,101.0,134.0
1,2022-05-11,0,Router 64,1460,0,33.0,76.0,101.0,2.142857,0.0,...,7.0,10.0,30.000000,59.0,32.0,65.0,102.0,61.0,101.0,134.0
2,2022-05-12,0,Router 64,1460,0,44.0,76.0,101.0,2.142857,0.0,...,7.0,10.0,30.000000,59.0,32.0,65.0,102.0,61.0,101.0,134.0
3,2022-05-13,0,Router 64,1460,10,34.0,66.0,92.0,3.285714,0.0,...,7.0,10.0,30.000000,59.0,32.0,65.0,102.0,61.0,101.0,134.0
4,2022-05-14,0,Router 64,1460,6,33.0,66.0,92.0,4.000000,0.0,...,7.0,10.0,30.000000,59.0,32.0,65.0,102.0,61.0,101.0,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,2022-06-05,78,Microphone 391,1650,5,18.0,79.0,159.0,4.285714,0.0,...,9.0,0.0,23.272727,57.0,12.0,51.0,107.0,29.0,76.0,154.0
2383,2022-06-06,78,Microphone 391,1650,0,18.0,86.0,176.0,4.285714,0.0,...,9.0,0.0,23.272727,57.0,12.0,51.0,107.0,29.0,76.0,154.0
2384,2022-06-07,78,Microphone 391,1650,0,26.0,100.0,188.0,3.571429,0.0,...,9.0,0.0,23.272727,57.0,12.0,51.0,107.0,29.0,76.0,154.0
2385,2022-06-08,78,Microphone 391,1650,0,26.0,100.0,203.0,2.142857,0.0,...,9.0,0.0,23.054545,57.0,12.0,51.0,107.0,29.0,76.0,154.0


1. 0.1 квантиль – "пессимистичный" прогноз (в 90% случаев прогноз ниже реальных продаж);
2. 0.5 квантиль – "консервативный" прогноз (в 50% случаев реальные продажи выше, в 50% ниже);
3. 0.9 квантиль – "оптимистичный" прогноз (в 90% прогноз окажется выше реальных продаж).

In [ ]:
df_merged.columns

Index(['day', 'sku_id', 'sku', 'price', 'qty', 'next_7d', 'next_14d',
       'next_21d', 'qty_7d_avg', 'qty_7d_q10', 'qty_7d_q50', 'qty_7d_q90',
       'qty_14d_avg', 'qty_14d_q10', 'qty_14d_q50', 'qty_14d_q90',
       'qty_21d_avg', 'qty_21d_q10', 'qty_21d_q50', 'qty_21d_q90',
       'pred_7d_q10', 'pred_7d_q50', 'pred_7d_q90', 'pred_14d_q10',
       'pred_14d_q50', 'pred_14d_q90', 'pred_21d_q10', 'pred_21d_q50',
       'pred_21d_q90'],
      dtype='object')

#7 ДНЕЙ

In [ ]:
missed_profit_pes7 = week_missed_profits(df=df_merged, sales_col='next_7d', forecast_col='pred_7d_q10', date_col='day', price_col='price')
missed_profit_pes7

,day,revenue,missed_profits
0,2022-05-15,10819150.0,17380.0
1,2022-05-22,11369990.0,39180.0
2,2022-05-29,11241470.0,28830.0
3,2022-06-05,11545520.0,6760.0
4,2022-06-12,9316120.0,0.0


In [ ]:
missed_profit_bas7 = week_missed_profits(df=df_merged, sales_col='next_7d', forecast_col='pred_7d_q50', date_col='day', price_col='price')
missed_profit_bas7

,day,revenue,missed_profits
0,2022-05-15,10819150.0,1.442471e+06
1,2022-05-22,11369990.0,1.387455e+06
2,2022-05-29,11241470.0,1.421366e+06
3,2022-06-05,11545520.0,1.374242e+06
4,2022-06-12,9316120.0,2.572520e+05


In [ ]:
missed_profit_opt7 = week_missed_profits(df=df_merged, sales_col='next_7d', forecast_col='pred_7d_q90', date_col='day', price_col='price')
missed_profit_opt7

,day,revenue,missed_profits
0,2022-05-15,10819150.0,1.410074e+07
1,2022-05-22,11369990.0,1.696396e+07
2,2022-05-29,11241470.0,1.699614e+07
3,2022-06-05,11545520.0,1.672346e+07
4,2022-06-12,9316120.0,7.454140e+06


#14 ДНЕЙ

In [ ]:
#cуммарное количество продаж на 14 дней
missed_profit_pes14 = week_missed_profits(df=df_merged, sales_col='next_14d', forecast_col='pred_14d_q10', date_col='day', price_col='price')
missed_profit_pes14

,day,revenue,missed_profits
0,2022-05-15,20671200.0,64750.0
1,2022-05-22,22611460.0,36040.0
2,2022-05-29,22786990.0,5270.0
3,2022-06-05,28634430.0,1570.0
4,2022-06-12,19161990.0,0.0


# 21 ДЕНЬ

In [ ]:
missed_profit_pes21 = week_missed_profits(df=df_merged, sales_col='next_21d', forecast_col='pred_21d_q10', date_col='day', price_col='price')
missed_profit_pes21

,day,revenue,missed_profits
0,2022-05-15,30103530.0,50160.0
1,2022-05-22,34156980.0,29210.0
2,2022-05-29,39875900.0,0.0
3,2022-06-05,45798510.0,69460.0
4,2022-06-12,28968970.0,34730.0


В отчет выводятся две оценки: в абсолютных и относительных величинах. Относительная величина считается относительно среднего значения фактической выручки в неделю.

1. Для каждой случайной выборки вычисляется среднее missed_profits и revenue
2. Строим доверительный интервал

In [ ]:
import numpy as np
from typing import Tuple

def missed_profits_ci(
    df: pd.DataFrame,
    missed_profits_col: str,
    confidence_level: float = 0.95,
    n_bootstraps: int = 1000,
) -> Tuple[Tuple[float, Tuple[float, float]], Tuple[float, Tuple[float, float]]]:
    """
    Estimates the missed profits for the given DataFrame.
    Calculates average missed_profits per week and estimates the confidence interval.
    """
    rng = np.random.default_rng()
    missed = df[missed_profits_col].values
    revenues = df["revenue"].values

    means_abs = []
    means_rel = []

    for _ in range(n_bootstraps):
        idx = rng.integers(0, len(df), len(df))
        boot_missed = missed[idx]
        boot_revenue = revenues[idx]
        mean_missed = boot_missed.mean()
        mean_revenue = boot_revenue.mean()
        means_abs.append(mean_missed)
        means_rel.append(mean_missed / mean_revenue if mean_revenue > 0 else np.nan)

    # Центральная оценка по исходным данным
    mean_abs = missed.mean()
    mean_rel = mean_abs / revenues.mean()

    # доверительный интервал
    alpha = (1 - confidence_level) / 2
    lower = int(n_bootstraps * alpha)
    upper = int(n_bootstraps * (1 - alpha))

    abs_ci = np.sort(means_abs)[[lower, upper]]
    rel_ci = np.sort(means_rel)[[lower, upper]]

    # Привести к tuple с немагическими числами
    return (
        (mean_abs, (abs_ci[0], abs_ci[1])),
        (mean_rel, (rel_ci[0], rel_ci[1])),
    )

**В 90% случаев фактические продажи выше этого прогноза**

In [ ]:
import numpy as np
ci = missed_profits_ci(missed_profit_pes7, 'missed_profits')
print('Среднее упущенной прибыли за неделю:', ci[0][0])
print('Доверительный интервал (абс):', ci[0][1])
print('Среднее % упущенной прибыли по выручке:', ci[1][0])
print('Доверительный интервал (отн):', ci[1][1])

Среднее упущенной прибыли за неделю: 18430.0
Доверительный интервал (абс): (np.float64(6180.0), np.float64(30900.0))
Среднее % упущенной прибыли по выручке: 0.001697295654536329
Доверительный интервал (отн): (np.float64(0.0005880961348761373), np.float64(0.002742480057554805))


В 50% случаев продажи выше прогноза.
В 50% случаев – ниже.

In [ ]:
import numpy as np
ci = missed_profits_ci(missed_profit_bas7, 'missed_profits')
print('Среднее упущенной прибыли за неделю:', ci[0][0])
print('Доверительный интервал (абс):', ci[0][1])
print('Среднее % упущенной прибыли по выручке:', ci[1][0])
print('Доверительный интервал (отн):', ci[1][1])

Среднее упущенной прибыли за неделю: 1176557.1784572555
Доверительный интервал (абс): (np.float64(713472.7798993265), np.float64(1423025.920593523))
Среднее % упущенной прибыли по выручке: 0.10835406328318088
Доверительный интервал (отн): (np.float64(0.07031318202193604), np.float64(0.12822079458263252))


**В 90% случаев фактические продажи ниже этого прогноза.
В 10% случаев продажи превышают прогноз (лучший сценарий).**

In [ ]:
import numpy as np
ci = missed_profits_ci(missed_profit_opt7, 'missed_profits')
print('Среднее упущенной прибыли за неделю:', ci[0][0])
print('Доверительный интервал (абс):', ci[0][1])
print('Среднее % упущенной прибыли по выручке:', ci[1][0])
print('Доверительный интервал (отн):', ci[1][1])

Среднее упущенной прибыли за неделю: 14447689.805853223
Доверительный интервал (абс): (np.float64(10691860.672210092), np.float64(16928734.35578298))
Среднее % упущенной прибыли по выручке: 1.330548080605724
Доверительный интервал (отн): (np.float64(1.0689940758849803), np.float64(1.4910339579335787))


**Training Pipeline**

In [ ]:
"""
 {
  # Анна Рожкова's workspace
  web_server: https://app.clear.ml/
  api_server: https://api.clear.ml
  files_server: https://files.clear.ml
  credentials {
    "access_key" = "IZH03SVDDS1XFN0RQN8A3EZ3OKUWXN"
    "secret_key" = "uD9fSEidYd90kyJr9llA8x1uZXG1Akl_IGfeFRVNfviNrcq4I1gRVxCXOLZMrkJctqM"
  }
}
"""

'\n {\n  # Анна Рожкова\'s workspace\n  web_server: https://app.clear.ml/\n  api_server: https://api.clear.ml\n  files_server: https://files.clear.ml\n  credentials {\n    "access_key" = "IZH03SVDDS1XFN0RQN8A3EZ3OKUWXN"\n    "secret_key" = "uD9fSEidYd90kyJr9llA8x1uZXG1Akl_IGfeFRVNfviNrcq4I1gRVxCXOLZMrkJctqM"\n  }\n}\n'